In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import tdt

# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
from acr.utils import *
bands = sp.bands

import pingouin as pg
from scipy import stats
from scipy.stats import shapiro

from acr.plots import pub, lrg
lrg()
plt.rcParams['xtick.bottom'] = False

import matplotlib as mpl
import brokenaxes as bx

#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')

In [ ]:
notebook_figure_root = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys'

In [ ]:
som_subjects = ['OPTSOM_1', 'OPTSOM_2', 'OPTSOM_3', 'OPTSOM_4', 'OPTSOM_5', 'OPTSOM_7', 'OPTSOM_8', 'OPTSOM_9']
acr_subjects = ['OPT_2', 'OPT_4', 'OPT_10', 'OPT_11', 'OPT_12', 'OPT_15', 'OPT_16']
control_subjects = ['OPT_5_CTRL', 'OPT_6_CTRL', 'OPT_8_CTRL', 'OPT_9_CTRL', 'OPT_13_CTRL', 'OPT_14_CTRL', 'OPT_17_CTRL', 'OPT_18_CTRL']

In [ ]:
master_fig_size = (3, 4)

# Custom Functions

In [ ]:
def get_molecular_stim_times(subject, recording, store='Pu1/'):
    raw_data_root = '/Volumes/em_storage/Electron_Microscopy/Rest-activity/OPT_EXP/OPTSOM_EXP' if 'OPTSOM' in subject else '/Volumes/em_storage/Electron_Microscopy/Rest-activity/OPT_EXP/ACR_EXP'
    block_path = f'{raw_data_root}/{subject}/{subject}-{recording}'
    data = tdt.read_block(block_path, store=store)
    onsets = data.epocs.Pu1_.onset
    offsets = data.epocs.Pu1_.offset
    onsets = pd.to_timedelta(onsets, unit='s')

    offsets = pd.to_timedelta(offsets, unit='s')
    rec_start = rec_start_time(subject, recording)
    onsets = onsets + rec_start
    offsets = offsets + rec_start
    return onsets, offsets

def rec_start_time(subject, recording):
    raw_data_root = '/Volumes/em_storage/Electron_Microscopy/Rest-activity/OPT_EXP/OPTSOM_EXP' if 'OPTSOM' in subject else '/Volumes/em_storage/Electron_Microscopy/Rest-activity/OPT_EXP/ACR_EXP'
    block_path = f'{raw_data_root}/{subject}/{subject}-{recording}'
    dat = tdt.read_block(block_path, evtype=['epocs'])
    return pd.Timestamp(dat.info.start_date)

def load_molecular_exp_hypno(subject, start_time):
    path = f'/Volumes/em_storage/Electron_Microscopy/Rest-activity/config_files_25/FINAL_HYPNOS/hypno_{subject}-exp.txt'
    hypno = kde.hypno.hypno.load_hypno_file(path, start_time, dt=True)
    return hypno

def read_mephys_data(subject, recording, exp_hypno=False):
    raw_data_root = '/Volumes/em_storage/Electron_Microscopy/Rest-activity/OPT_EXP/OPTSOM_EXP' if 'OPTSOM' in subject else '/Volumes/em_storage/Electron_Microscopy/Rest-activity/OPT_EXP/ACR_EXP'
    block_path = f'{raw_data_root}/{subject}/{subject}-{recording}'
    dat = kde.xr.io.get_data(block_path, store='EEGr', channel=[1, 2])
    start_time = dat.datetime.values[0]
    if exp_hypno:
        hypno = load_molecular_exp_hypno(subject, start_time)
        return dat, hypno
    else:
        return dat
def label_df_between_times(df, ons, offs, label='stim'):
    if 'label' not in df.columns:
        df['label'] = "NO_LABEL"
    for i in range(len(ons)):
        df.loc[(df['datetime'] >= ons[i]) & (df['datetime'] <= offs[i]), 'label'] = label
    return df

def label_bl(df, stim_start, label='bl'):
    df.loc[df['datetime'] < stim_start, 'label'] = label
    return df


In [ ]:
def delta_rel_to_bl_ts(df, state='Wake'):
    """df should have a label column with only two values: 'bl' and 'stim'

    Parameters
    ----------
    df : _type_
        _description_

    """
    if state is not None:
        state_df = df.loc[df['state'] == state]
    else:
        state_df = df
    means = state_df.groupby(['label', 'channel']).mean().reset_index()
    means = means.loc[means['label'] == 'bl']
    means.drop(columns=['time'], inplace=True)
    df['delta_bl'] = 0
    df.loc[df['channel'] == 1, 'delta_bl'] = means.loc[means['channel'] == 1, 'delta'].values[0]
    df.loc[df['channel'] == 2, 'delta_bl'] = means.loc[means['channel'] == 2, 'delta'].values[0]
    df['delta_rel'] = df['delta']/df['delta_bl']
    return df

In [ ]:
def delta_rel_to_bl(df, state='Wake'):
    """df should have a label column with only two values: 'bl' and 'stim'

    Parameters
    ----------
    df : _type_
        _description_

    """
    state_df = df.loc[df['state'] == state] 
    means = state_df.groupby(['label', 'channel']).mean().reset_index()
    means = means.loc[means['label'] == 'bl']
    means.drop(columns=['time'], inplace=True)
    df['delta_bl'] = 0
    df.loc[df['channel'] == 1, 'delta_bl'] = means.loc[means['channel'] == 1, 'delta'].values[0]
    df.loc[df['channel'] == 2, 'delta_bl'] = means.loc[means['channel'] == 2, 'delta'].values[0]
    df['delta_rel'] = df['delta']/df['delta_bl']
    return df

# Control Subjects

In [ ]:
bps = {}
h = {}
for subject in control_subjects:
    dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
    h[subject] = hypno
    spg = kde.xr.spectral.get_spextrogram(dat)
    bp = kde.xr.spectral.get_bp_set(spg)
    bp = kde.hypno.hypno.add_states(bp, hypno)
    bps[subject] = bp
bpdf = []
for subject in bps.keys():
    df = bps[subject].delta.to_dataframe().reset_index()
    df['subject'] = subject
    ons, offs = get_molecular_stim_times(subject, 'swi')
    ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
    df = label_df_between_times(df, ton, toff, label='stim')
    df = label_bl(df, ton[0])
    df = df.loc[df['label'] != 'NO_LABEL']
    df = df.loc[df['state'] == 'Wake']
    delta_rel_to_bl(df, state='Wake')
    bpdf.append(df)
bpdf = pd.concat(bpdf)

In [ ]:
#bpw = bpdf.loc[bpdf['state'] == 'Wake']
delta_means = bpdf.groupby(['subject', 'channel', 'label']).mean().reset_index()

In [ ]:
dm = delta_means.loc[delta_means['label'] == 'stim']
ch1 = dm.chnl(1)['delta_rel'].values
ch2 = dm.chnl(2)['delta_rel'].values
ch1_norm = ch1/ch2
ch2_norm = ch2/ch2

In [ ]:
f, ax = acr.plots.gen_paired_boxplot(ch2, ch1, colors=[NNXR_GRAY, NNXR_GRAY], fsize=(4, 3))
#f, ax = pub_utils.swa_boxplot(ch2, ch1, stats=False, one_sided=False, color=NNXR_GRAY, mean_size=28, fsize=(4, 3))

ax.set_ylim(0.85, 1.5) # y limits
ax.set_xlim(0.3, 0.70) # x limits
ax.set_yticks([0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5]) # y ticks

#plt.tight_layout(pad=0.2)

fig_name = 'MOLECULAR--control_subs_swa_during_stim_raw'
fig_path = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys/{fig_name}.png'
plt.savefig(fig_path, bbox_inches='tight', dpi=600, transparent=True)

In [ ]:
write = True

diffs = ch1 - ch2
_s, _p = shapiro(ch1-ch2)
print(f'Shapiro-Wilk p-value = {_p}')

stats = pg.ttest(ch1, ch2, paired=True)
hg = pg.compute_effsize(ch1, ch2, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

if write:
    # ==== Write Stats Results ====
    stats_name = fig_name
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # source_data
    source_dat = pd.DataFrame({'off_induction_eeg': ch1, 'contra_eeg': ch2})
    pub_utils.write_source_data(source_dat, fig_name)

stats


In [ ]:
f, ax = acr.plots.gen_paired_boxplot(ch2_norm, ch1_norm, colors=[NNXR_GRAY, NNXR_GRAY], fsize=(4, 3), one_sided=True)
ax.set_ylim(0.85, 1.3) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([0.9, 1, 1.1, 1.2, 1.3]) # y ticks

fig_name = 'MOLECULAR--control_subs_swa_during_stim_ReltoContra'
fig_path = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys/{fig_name}.png'
plt.savefig(fig_path, bbox_inches='tight', dpi=600, transparent=True)

# SOM SUBJECTS

In [ ]:
bps = {}
h = {}
for subject in som_subjects:
    dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
    h[subject] = hypno
    spg = kde.xr.spectral.get_spextrogram(dat)
    bp = kde.xr.spectral.get_bp_set(spg)
    bp = kde.hypno.hypno.add_states(bp, hypno)
    bps[subject] = bp

bpdf = []
for subject in bps.keys():
    df = bps[subject].delta.to_dataframe().reset_index()
    df['subject'] = subject
    ons, offs = get_molecular_stim_times(subject, 'swi')
    ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
    df = label_df_between_times(df, ton, toff, label='stim')
    df = label_bl(df, ton[0])
    df = df.loc[df['label'] != 'NO_LABEL']
    df = df.loc[df['state'] == 'Wake']
    df = delta_rel_to_bl(df)
    bpdf.append(df)
bpdf = pd.concat(bpdf)

In [ ]:
#bpw = bpdf.loc[bpdf['state'] == 'Wake']
delta_means = bpdf.groupby(['subject', 'channel', 'label']).mean().reset_index()

In [ ]:
dm = delta_means.loc[delta_means['label'] == 'stim']
ch1 = dm.chnl(1)['delta_rel'].values
ch2 = dm.chnl(2)['delta_rel'].values
ch1_norm = ch1/ch2
ch2_norm = ch2/ch2

In [ ]:
f, ax = acr.plots.gen_paired_boxplot(ch2, ch1, colors=[NNXR_GRAY, SOM_BLUE], fsize=(4, 3))
#f, ax = pub_utils.swa_boxplot(ch2, ch1, stats=False, one_sided=False, color=NNXR_GRAY, mean_size=28, fsize=(4, 3))

ax.set_ylim(1.0, 7) # y limits
ax.set_xlim(0.3, 0.70) # x limits
ax.set_yticks([1, 2, 3, 4, 5, 6, 7]) # y ticks

#plt.tight_layout(pad=0.2)

fig_name = 'MOLECULAR--SOM_subs_swa_during_stim_raw'
fig_path = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys/{fig_name}.png'
plt.savefig(fig_path, bbox_inches='tight', dpi=600, transparent=True)

In [ ]:
write = True

diffs = ch1 - ch2
_s, _p = shapiro(ch1-ch2)
print(f'Shapiro-Wilk p-value = {_p}')

stats = pg.ttest(ch1, ch2, paired=True)
hg = pg.compute_effsize(ch1, ch2, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

if write:
    # ==== Write Stats Results ====
    stats_name = fig_name
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # source_data
    source_dat = pd.DataFrame({'off_induction_eeg': ch1, 'contra_eeg': ch2})
    pub_utils.write_source_data(source_dat, fig_name)

stats


In [ ]:
f, ax = acr.plots.gen_paired_boxplot(ch2_norm, ch1_norm, colors=[NNXR_GRAY, SOM_BLUE], fsize=(4, 3), one_sided=True)

ax.set_ylim(0.9, 2.2) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([1, 1.5, 2]) # y ticks

fig_name = 'MOLECULAR--SOM_subs_swa_during_stim_ReltoContra'
fig_path = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys/{fig_name}.png'
plt.savefig(fig_path, bbox_inches='tight', dpi=600, transparent=True)

# ACR ANIMALS

In [ ]:
import xarray as xr

In [ ]:
bps = {}
h = {}
datsd = read_mephys_data('OPT_1', 'swi-sd')
dat = read_mephys_data('OPT_1', 'swi')
dat1 = xr.concat([datsd, dat], dim='datetime')
hsd = kde.hypno.hypno.load_hypno_file(f'/Volumes/em_storage/Electron_Microscopy/Rest-activity/config_files_25/FINAL_HYPNOS/hypno_OPT_1-exp1.txt', st=datsd.datetime.values[0], dt=True)
hexp = kde.hypno.hypno.load_hypno_file(f'/Volumes/em_storage/Electron_Microscopy/Rest-activity/config_files_25/FINAL_HYPNOS/hypno_OPT_1-exp2.txt', st=dat.datetime.values[0], dt=True)
h1 = pd.concat([hsd, hexp])
h['OPT_1'] = h1
spg = kde.xr.spectral.get_spextrogram(dat1)
bp = kde.xr.spectral.get_bp_set(spg)
bp = kde.hypno.hypno.add_states(bp, h1)
bp = bp.swap_dims({'channel': 'channel'}).assign_coords(channel=[2,1])
bps['OPT_1'] = bp

In [ ]:
for subject in acr_subjects:
    dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
    h[subject] = hypno
    spg = kde.xr.spectral.get_spextrogram(dat)
    bp = kde.xr.spectral.get_bp_set(spg)
    bp = kde.hypno.hypno.add_states(bp, hypno)
    bps[subject] = bp

bpdf = []
for subject in bps.keys():
    df = bps[subject].delta.to_dataframe().reset_index()
    df['subject'] = subject
    ons, offs = get_molecular_stim_times(subject, 'swi')
    ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
    df = label_df_between_times(df, ton, toff, label='stim')
    df = label_bl(df, ton[0])
    df = df.loc[df['label'] != 'NO_LABEL']
    df = df.loc[df['state'] == 'Wake']
    df = delta_rel_to_bl(df)
    bpdf.append(df)
bpdf = pd.concat(bpdf)

In [ ]:
#bpw = bpdf.loc[bpdf['state'] == 'Wake']
delta_means = bpdf.groupby(['subject', 'channel', 'label']).mean().reset_index()

In [ ]:
dm = delta_means.loc[delta_means['label'] == 'stim']
ch1 = dm.chnl(1)['delta_rel'].values
ch2 = dm.chnl(2)['delta_rel'].values
ch1_norm = ch1/ch2
ch2_norm = ch2/ch2

In [ ]:
f, ax = acr.plots.gen_paired_boxplot(ch2, ch1, colors=[NNXR_GRAY, ACR_BLUE], fsize=(4, 3))
#f, ax = pub_utils.swa_boxplot(ch2, ch1, stats=False, one_sided=False, color=NNXR_GRAY, mean_size=28, fsize=(4, 3))

ax.set_ylim(0, 25) # y limits
ax.set_xlim(0.3, 0.70) # x limits
ax.set_yticks([0, 10,  20]) # y ticks

#plt.tight_layout(pad=0.2)

fig_name = 'MOLECULAR--ACR_subs_swa_during_stim_raw'
fig_path = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys/{fig_name}.png'
plt.savefig(fig_path, bbox_inches='tight', dpi=600, transparent=True)

In [ ]:
write = True

diffs = ch1 - ch2
_s, _p = shapiro(ch1-ch2)
print(f'Shapiro-Wilk p-value = {_p}')

stats = pg.ttest(ch1, ch2, paired=True)
hg = pg.compute_effsize(ch1, ch2, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

if write:
    # ==== Write Stats Results ====
    stats_name = fig_name
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # source_data
    source_dat = pd.DataFrame({'off_induction_eeg': ch1, 'contra_eeg': ch2})
    pub_utils.write_source_data(source_dat, fig_name)

stats


In [ ]:
f, ax = acr.plots.gen_paired_boxplot(ch2_norm, ch1_norm, colors=[NNXR_GRAY, ACR_BLUE], fsize=(4, 3), one_sided=True)

ax.set_ylim(0.65, 9) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([1, 5, 9]) # y ticks

fig_name = 'MOLECULAR--ACR_subs_swa_during_stim_ReltoContra'
fig_path = f'{pub_utils.PAPER_FIGURE_ROOT}/molecular/ephys/{fig_name}.png'
plt.savefig(fig_path, bbox_inches='tight', dpi=600, transparent=True)

# All subjects averaged SWA over time

In [ ]:
sub_dfs = {}
for sub_type, sub_list in zip(['som', 'acr', 'control'], [som_subjects, acr_subjects, control_subjects]):
    for subject in sub_list:
        dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
        spg = kde.xr.spectral.get_spextrogram(dat)
        bp = kde.xr.spectral.get_bp_set(spg)
        bp = kde.hypno.hypno.add_states(bp, hypno)

        df = bp.delta.to_dataframe().reset_index()
        df['subject'] = subject
        ons, offs = get_molecular_stim_times(subject, 'swi')
        ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
        df = label_df_between_times(df, ton, toff, label='stim')
        df = label_bl(df, ton[0])
        df = delta_rel_to_bl_ts(df, state=None) # AVERAGE OVER THE WHOLE BASELINE!
        df['type'] = sub_type
        sub_dfs[subject] = df

In [ ]:
start = "09:00:00"
end = "13:50:00"
divisions = pd.date_range(start=start, end=end, freq='10min')
new_dfs = []
for subject in sub_dfs.keys():
    df = sub_dfs[subject]
    df['epoch'] = 'NA'
    day = str(df['datetime'].min()).split(' ')[0]
    for i, d in enumerate(divisions):
        d_time = str(d).split(' ')[1]
        t1 = pd.Timestamp(day + ' ' + d_time)
        t2 = t1 + pd.Timedelta('10min')
        df.loc[(df['datetime'] >= t1) & (df['datetime'] < t2), 'epoch'] = i
    df = df.loc[df['epoch'] != 'NA']
    new_dfs.append(df)
all_dfs = pd.concat(new_dfs)

In [ ]:
som_df = all_dfs.loc[all_dfs['type'] == 'som']
acr_df = all_dfs.loc[all_dfs['type'] == 'acr']
con_df = all_dfs.loc[all_dfs['type'] == 'control']
type_dfs = {'som': som_df, 'acr': acr_df, 'control': con_df}

In [ ]:
type_dfs['som'] = type_dfs['som'].loc[type_dfs['som']['subject'] != 'OPTSOM_8']

In [ ]:
means = {}
sems = {}
ep_means = {}
for type in type_dfs.keys():
    df = type_dfs[type]
    epoch_means = df.groupby(['subject', 'channel', 'epoch']).mean().reset_index()
    ep_means[type] = epoch_means
    epochs=epoch_means.chnl(2).epoch.unique()
    ch2 =epoch_means.chnl(2).sort_values(['subject', 'epoch']).delta_rel.values
    ch1 = epoch_means.chnl(1).sort_values(['subject', 'epoch'])
    ch1['drel_ch2'] = ch2
    ch1['drel_norm'] = ch1['delta_rel']/ch1['drel_ch2']
    norm_means = ch1.groupby('epoch').mean().drel_norm.values
    norm_stds = ch1.groupby('epoch').sem().drel_norm.values
    means[type] = norm_means
    sems[type] = norm_stds

In [ ]:
plt.rcdefaults()
lrg()

In [ ]:
f, ax = plt.subplots(figsize=(30, 10))
for type, color in zip(type_dfs.keys(), [SOM_BLUE, ACR_BLUE, NNXR_GRAY]):
        ax.errorbar(epochs,
                means[type],
                yerr=sems[type],
                color=color, 
                linewidth=0, 
                marker='o', 
                markersize=10, 
                markerfacecolor=color, 
                markeredgecolor=color,
                capsize=4,
                capthick=3.5, 
                elinewidth=3.5, 
                ecolor=color, 
                alpha=0.9,  # This alpha applies to the error bars
                zorder=1)

        # Add a separate line plot for the connecting line with different alpha
        ax.plot(epochs, means[type], 
                color=color, 
                linewidth=4.5, 
                alpha=0.6)  # Adjust this value for the desired line transparency
ax.set_yticks([0, 2, 4, 6])
ax.set_ylim(0, 6)
ax.set_xticks([0, 6, 12, 18, 24, 30])

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
sns.lineplot(data=ep_means['acr'].chnl(2), x='epoch', y='delta_rel', ax=ax, errorbar='se', color='gray', linewidth=4)
sns.lineplot(data=ep_means['acr'].chnl(1), x='epoch', y='delta_rel', ax=ax, errorbar='se', color=ACR_BLUE, linewidth=4)
ax.set_yticks([0, 2, 4, 6, 8, 10, 12])
ax.set_ylim(0, 12)
ax.set_xticks([0, 6, 12, 18, 24, 30])

In [ ]:
epochs = epochs + 1

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
color=ACR_BLUE
sub_type = 'acr'

ax.errorbar(epochs,
        ep_means[sub_type].chnl(1).groupby('epoch').mean().delta_rel.values,
        yerr=ep_means[sub_type].chnl(1).groupby('epoch').sem().delta_rel.values,
        color=color, 
        linewidth=0, 
        marker='o', 
        markersize=10, 
        markerfacecolor=color, 
        markeredgecolor=color,
        capsize=4,
        capthick=3.5, 
        elinewidth=3.5, 
        ecolor=color, 
        alpha=0.9,  # This alpha applies to the error bars
        zorder=1)

# Add a separate line plot for the connecting line with different alpha
ax.plot(epochs, ep_means[sub_type].chnl(1).groupby('epoch').mean().delta_rel.values, 
        color=color, 
        linewidth=4.5, 
        alpha=0.6)  # Adjust this value for the desired line transparency


ax.errorbar(epochs,
        ep_means[sub_type].chnl(2).groupby('epoch').mean().delta_rel.values,
        yerr=ep_means[sub_type].chnl(2).groupby('epoch').sem().delta_rel.values,
        color='gray', 
        linewidth=0, 
        marker='o', 
        markersize=10, 
        markerfacecolor='gray', 
        markeredgecolor='gray',
        capsize=4,
        capthick=3.5, 
        elinewidth=3.5, 
        ecolor='gray', 
        alpha=0.9,  # This alpha applies to the error bars
        zorder=1)

# Add a separate line plot for the connecting line with different alpha
ax.plot(epochs, ep_means[sub_type].chnl(2).groupby('epoch').mean().delta_rel.values, 
        color='gray', 
        linewidth=4.5, 
        alpha=0.6)  # Adjust this value for the desired line transparency

ax.set_yticks([0, 4, 8, 12])
ax.set_ylim(0, 12)
ax.set_xticks([0, 6, 12, 18, 24, 30])
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/{sub_type}_subs_swa_over_SD__rel_to_full_SDmean.png', bbox_inches='tight', dpi=600, transparent=True)

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
sns.lineplot(data=ep_means['som'].chnl(2), x='epoch', y='delta_rel', ax=ax, errorbar='se', color='gray', linewidth=4)
sns.lineplot(data=ep_means['som'].chnl(1), x='epoch', y='delta_rel', ax=ax, errorbar='se', color=SOM_BLUE, linewidth=4)

ax.set_yticks([0.5, 1, 2, 3])
ax.set_ylim(0.5, 3)
ax.set_xticks([0, 6, 12, 18, 24, 30])

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
color=SOM_BLUE
sub_type = 'som'

ax.errorbar(epochs,
        ep_means[sub_type].chnl(1).groupby('epoch').mean().delta_rel.values,
        yerr=ep_means[sub_type].chnl(1).groupby('epoch').sem().delta_rel.values,
        color=color, 
        linewidth=0, 
        marker='o', 
        markersize=10, 
        markerfacecolor=color, 
        markeredgecolor=color,
        capsize=4,
        capthick=3.5, 
        elinewidth=3.5, 
        ecolor=color, 
        alpha=0.9,  # This alpha applies to the error bars
        zorder=1)

# Add a separate line plot for the connecting line with different alpha
ax.plot(epochs, ep_means[sub_type].chnl(1).groupby('epoch').mean().delta_rel.values, 
        color=color, 
        linewidth=4.5, 
        alpha=0.6)  # Adjust this value for the desired line transparency


ax.errorbar(epochs,
        ep_means[sub_type].chnl(2).groupby('epoch').mean().delta_rel.values,
        yerr=ep_means[sub_type].chnl(2).groupby('epoch').sem().delta_rel.values,
        color='gray', 
        linewidth=0, 
        marker='o', 
        markersize=10, 
        markerfacecolor='gray', 
        markeredgecolor='gray',
        capsize=4,
        capthick=3.5, 
        elinewidth=3.5, 
        ecolor='gray', 
        alpha=0.9,  # This alpha applies to the error bars
        zorder=1)

# Add a separate line plot for the connecting line with different alpha
ax.plot(epochs, ep_means[sub_type].chnl(2).groupby('epoch').mean().delta_rel.values, 
        color='gray', 
        linewidth=4.5, 
        alpha=0.6)  # Adjust this value for the desired line transparency

ax.set_yticks([0.5, 1, 2, 3])
ax.set_ylim(0.5, 3)
ax.set_xticks([0, 6, 12, 18, 24, 30])
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/{sub_type}_subs_swa_over_SD__rel_to_full_SDmean.png', bbox_inches='tight', dpi=600, transparent=True)

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
sns.lineplot(data=ep_means['control'].chnl(2), x='epoch', y='delta_rel', ax=ax, errorbar='se', color='gray', linewidth=4)
sns.lineplot(data=ep_means['control'].chnl(1), x='epoch', y='delta_rel', ax=ax, errorbar='se', color='black', linewidth=4)
ax.set_yticks([0.5, 1, 1.5, 2])
ax.set_ylim(0.5, 2)
ax.set_xticks([0, 6, 12, 18, 24, 30])

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
color='black'
sub_type = 'control'

ax.errorbar(epochs,
        ep_means[sub_type].chnl(1).groupby('epoch').mean().delta_rel.values,
        yerr=ep_means[sub_type].chnl(1).groupby('epoch').sem().delta_rel.values,
        color=color, 
        linewidth=0, 
        marker='o', 
        markersize=10, 
        markerfacecolor=color, 
        markeredgecolor=color,
        capsize=4,
        capthick=3.5, 
        elinewidth=3.5, 
        ecolor=color, 
        alpha=0.9,  # This alpha applies to the error bars
        zorder=1)

# Add a separate line plot for the connecting line with different alpha
ax.plot(epochs, ep_means[sub_type].chnl(1).groupby('epoch').mean().delta_rel.values, 
        color=color, 
        linewidth=4.5, 
        alpha=0.6)  # Adjust this value for the desired line transparency


ax.errorbar(epochs,
        ep_means[sub_type].chnl(2).groupby('epoch').mean().delta_rel.values,
        yerr=ep_means[sub_type].chnl(2).groupby('epoch').sem().delta_rel.values,
        color='gray', 
        linewidth=0, 
        marker='o', 
        markersize=10, 
        markerfacecolor='gray', 
        markeredgecolor='gray',
        capsize=4,
        capthick=3.5, 
        elinewidth=3.5, 
        ecolor='gray', 
        alpha=0.9,  # This alpha applies to the error bars
        zorder=1)

# Add a separate line plot for the connecting line with different alpha
ax.plot(epochs, ep_means[sub_type].chnl(2).groupby('epoch').mean().delta_rel.values, 
        color='gray', 
        linewidth=4.5, 
        alpha=0.6)  # Adjust this value for the desired line transparency

ax.set_yticks([0.5, 1, 1.5, 2])
ax.set_ylim(0.5, 2)
ax.set_xticks([0, 6, 12, 18, 24, 30])
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/{sub_type}_subs_swa_over_SD__rel_to_full_SDmean.png', bbox_inches='tight', dpi=600, transparent=True)

# Time Series Plots for Schematic

## ACR

In [ ]:
bps = {}
h = {}
subject = 'OPT_12'
dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
spg = kde.xr.spectral.get_spextrogram(dat)
bp = kde.xr.spectral.get_bp_set(spg)
bp = kde.hypno.hypno.add_states(bp, hypno)

df = bp.delta.to_dataframe().reset_index()
df['subject'] = subject
ons, offs = get_molecular_stim_times(subject, 'swi')
ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
df = label_df_between_times(df, ton, toff, label='stim')
df = label_bl(df, ton[0])
df = delta_rel_to_bl_ts(df)

In [ ]:

ch1 = df.chnl(1)
ch2 = df.chnl(2)
ch1['delta_smoothed'] = ch1.sm('delta', sigma=4)['delta'].values
ch2['delta_smoothed'] = ch2.sm('delta', sigma=4)['delta'].values

In [ ]:
plt.rcdefaults()
plt.style.use('fivethirtyeight')
f, ax = plt.subplots(figsize=(30, 10))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color=SOM_BLUE)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9)
ax.axhline(1, color='red', linewidth=2)

In [ ]:
# set obviously artifactual values to median
ch1.loc[(ch1['delta_smoothed']>8000) & (ch1['label'] != 'stim'), 'delta_smoothed'] = np.median(ch1['delta_smoothed'])
ch2.loc[(ch2['delta_smoothed']>8000) & (ch2['label'] != 'stim'), 'delta_smoothed'] = np.median(ch2['delta_smoothed'])

# PLOT
plt.rcdefaults()
pub()
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

f, ax = plt.subplots(figsize=(30, 8))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color=ACR_BLUE, linewidth=6)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=1, linewidth=6)
ax.set_ylim(0, 70000)
ax.set_xlim(ch1['datetime'].values[0], ch1['datetime'].values[-1])
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
f.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/OPT_12_SWA-TS.png', bbox_inches='tight', dpi=600, transparent=True)

In [ ]:

# PLOT
plt.rcdefaults()
pub()
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

f, ax = plt.subplots(figsize=(30, 8))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color=ACR_BLUE, linewidth=6)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=1, linewidth=6)
ax.set_ylim(0, 70000)
ax.set_xlim(ch1['datetime'].values[0], ch1['datetime'].values[-1])
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
ax.axvspan(ton[0], ton[0]-pd.Timedelta('1h'), color='red', alpha=0.1)
ax.axhspan(10000, 20000, xmin=0.2, xmax=0.4,color='purple', alpha=0.1)
f.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/OPT_12_SWA-TS__SCALED-1h-10000uvhz.png', bbox_inches='tight', dpi=600, transparent=True)

## SOM

In [ ]:
bps = {}
h = {}
subject = 'OPTSOM_3'
dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
spg = kde.xr.spectral.get_spextrogram(dat)
bp = kde.xr.spectral.get_bp_set(spg)
bp = kde.hypno.hypno.add_states(bp, hypno)

df = bp.delta.to_dataframe().reset_index()
df['subject'] = subject
ons, offs = get_molecular_stim_times(subject, 'swi')
ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
df = label_df_between_times(df, ton, toff, label='stim')
df = label_bl(df, ton[0])
df = delta_rel_to_bl_ts(df)

In [ ]:

ch1 = df.chnl(1)
ch2 = df.chnl(2)
ch1['delta_smoothed'] = ch1.sm('delta', sigma=4)['delta'].values
ch2['delta_smoothed'] = ch2.sm('delta', sigma=4)['delta'].values

In [ ]:
plt.rcdefaults()
plt.style.use('fivethirtyeight')
f, ax = plt.subplots(figsize=(30, 10))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color=SOM_BLUE)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9)
ax.axhline(1, color='red', linewidth=2)

In [ ]:
# set obviously artifactual values to median
ch1.loc[(ch1['delta_smoothed']>6000) & (ch1['label'] != 'stim'), 'delta_smoothed'] = np.median(ch1['delta_smoothed'])
ch2.loc[(ch2['delta_smoothed']>6000) & (ch2['label'] != 'stim'), 'delta_smoothed'] = np.median(ch2['delta_smoothed'])

# PLOT
plt.rcdefaults()
pub()
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

f, ax = plt.subplots(figsize=(30, 5))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color=SOM_BLUE, linewidth=4)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9, linewidth=4)
ax.set_ylim(0, 25000)
ax.set_xlim(ch1['datetime'].values[0], ch1['datetime'].values[-1])
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
f.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/OPTSOM_3_SWA-TS.png', bbox_inches='tight', dpi=600, transparent=True)

In [ ]:

# PLOT
plt.rcdefaults()
pub()
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

f, ax = plt.subplots(figsize=(30, 5))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color=SOM_BLUE, linewidth=4)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9, linewidth=4)
ax.set_ylim(0, 25000)
ax.set_xlim(ch1['datetime'].values[0], ch1['datetime'].values[-1])
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
ax.axvspan(ton[0], ton[0]-pd.Timedelta('1h'), color='red', alpha=0.1)
ax.axhspan(10000, 20000, xmin=0.2, xmax=0.4,color='purple', alpha=0.1)
f.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/OPTSOM_3_SWA-TS__SCALED-1h-10000uvhz.png', bbox_inches='tight', dpi=600, transparent=True)

## Control

In [ ]:
bps = {}
h = {}
subject = 'OPT_9_CTRL'
dat, hypno = read_mephys_data(subject, 'swi', exp_hypno=True)
spg = kde.xr.spectral.get_spextrogram(dat)
bp = kde.xr.spectral.get_bp_set(spg)
bp = kde.hypno.hypno.add_states(bp, hypno)

df = bp.delta.to_dataframe().reset_index()
df['subject'] = subject
ons, offs = get_molecular_stim_times(subject, 'swi')
ton, toff = acr.stim.get_pulse_train_times(ons, offs, times=True)
df = label_df_between_times(df, ton, toff, label='stim')
df = label_bl(df, ton[0])
df = delta_rel_to_bl_ts(df)

In [ ]:
ch1 = df.chnl(1)
ch2 = df.chnl(2)
ch1['delta_smoothed'] = ch1.sm('delta', sigma=4)['delta'].values
ch2['delta_smoothed'] = ch2.sm('delta', sigma=4)['delta'].values

In [ ]:
plt.rcdefaults()
plt.style.use('fivethirtyeight')
f, ax = plt.subplots(figsize=(30, 10))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color='k')
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9)
ax.axhline(1, color='red', linewidth=2)
ax.set_ylim(0, 20000)

In [ ]:
# set obviously artifactual values to median
ch1.loc[(ch1['delta_smoothed']>5500), 'delta_smoothed'] = np.median(ch1['delta_smoothed'])
ch2.loc[(ch2['delta_smoothed']>5500), 'delta_smoothed'] = np.median(ch2['delta_smoothed'])

# PLOT
plt.rcdefaults()
pub()
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

f, ax = plt.subplots(figsize=(30, 5))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color='k', linewidth=4)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9, linewidth=4)
ax.set_ylim(0, 25000)
ax.set_xlim(ch1['datetime'].values[0], ch1['datetime'].values[-1])
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
f.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/OPT9_CTRL_SWA-TS.png', bbox_inches='tight', dpi=600, transparent=True)

In [ ]:

# PLOT
plt.rcdefaults()
pub()
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False

f, ax = plt.subplots(figsize=(30, 5))
ax.plot(ch1['datetime'].values, ch1['delta_smoothed'].values, color='k', linewidth=4)
ax.plot(ch2['datetime'].values, ch2['delta_smoothed'].values, color=NNXR_GRAY, alpha=0.9, linewidth=4)
ax.set_ylim(0, 25000)
ax.set_xlim(ch1['datetime'].values[0], ch1['datetime'].values[-1])
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
ax.axvspan(ton[0], ton[0]-pd.Timedelta('1h'), color='red', alpha=0.1)
ax.axhspan(10000, 20000, xmin=0.2, xmax=0.4,color='purple', alpha=0.1)
f.savefig(f'{PAPER_FIGURE_ROOT}/molecular/ephys/OPT9_CTRL_SWA-TS__SCALED-1h-10000uvhz.png', bbox_inches='tight', dpi=600, transparent=True)